# Gradinet Checking
Backpropagation is quite challenging to implement, and sometimes has bugs, Sometimes due to creticality of the application we wants to be really certain that our implementation of backpropagation is correct. To have this reassurance we need to use "gradient checking".

In [1]:
# Imports
import numpy as np
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector, gradient_check_test_params

In [2]:
# 1 Dimensonal Gradient Checking
def forward_propagation(x, theta):
    J = np.dot(theta, x)
    return J

In [3]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


In [4]:
def backward_propagation(x, theta):
    dtheta = x
    return x

In [5]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


In [12]:
def gradient_check(x, theta, epsilon = 1e-7):
    theta_plus = theta + epsilon
    theta_minus = theta - epsilon
    J_plus = np.dot(theta_plus, x)
    J_minus = np.dot(theta_minus , x)
    grad_approx = (J_plus - J_minus) / (2*epsilon)

    grad = x

    difference = np.linalg.norm(grad_approx - grad) / (np.linalg.norm(grad_approx) + np.linalg.norm(grad))

    if difference < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference

In [13]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

The gradient is correct!
difference = 2.919335883291695e-10
